# 03. Probability Calibration
## Logistic Regression for Rapid Spread Probability

![SYLVA](https://img.shields.io/badge/SYLVA-v1.0.0-blue)

This notebook demonstrates the calibration of RSI to rapid spread probability using logistic regression.

In [ ]:
# Import libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit

sys.path.insert(0, os.path.abspath('..'))

from sylva_fire.integration.probability_calibration import ProbabilityCalibrator
from sylva_fire.forecasting.rapid_spread_forecast import RapidSpreadForecaster

print("✅ Libraries imported")

## Probability Calibration Equation

$$P(RS) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 \cdot RSI + \beta_2 \cdot RSI^2 + \beta_3 \cdot C)}}$$

In [ ]:
# Display calibration coefficients by fuel type
fuel_types = ['pinus_halepensis', 'quercus_ilex', 'mediterranean_maquis', 'dry_grassland']

coeffs_data = []
for ft in fuel_types:
    calibrator = ProbabilityCalibrator(ft)
    coeffs_data.append({
        'Fuel Type': ft,
        'β₀': calibrator.coefficients['beta_0'],
        'β₁': calibrator.coefficients['beta_1'],
        'β₂': calibrator.coefficients['beta_2'],
        'β₃': calibrator.coefficients['beta_3']
    })

coeffs_df = pd.DataFrame(coeffs_data)
print("\n=== Calibration Coefficients ===\n")
print(coeffs_df.to_string(index=False))

In [ ]:
# Plot calibration curves for different fuel types
rsi_values = np.linspace(0, 1, 100)
confidence = 1.0

plt.figure(figsize=(12, 8))

for ft in fuel_types:
    calibrator = ProbabilityCalibrator(ft)
    probabilities = [calibrator.calibrate_probability(rsi, confidence) for rsi in rsi_values]
    plt.plot(rsi_values, probabilities, linewidth=2, label=ft)

plt.axhline(y=0.6, color='orange', linestyle='--', alpha=0.5, label='Warning threshold (60%)')
plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Imminent threshold (80%)')
plt.xlabel('Rapid Spread Index (RSI)')
plt.ylabel('Rapid Spread Probability')
plt.title('Probability Calibration Curves by Fuel Type')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

In [ ]:
# Effect of confidence on probability
calibrator = ProbabilityCalibrator('pinus_halepensis')
rsi_fixed = 0.7
confidence_levels = np.linspace(0.2, 1.0, 9)

probabilities = [calibrator.calibrate_probability(rsi_fixed, c) for c in confidence_levels]

plt.figure(figsize=(10, 6))
plt.plot(confidence_levels, probabilities, 'bo-', linewidth=2)
plt.xlabel('Confidence Factor')
plt.ylabel('Rapid Spread Probability')
plt.title(f'Effect of Confidence on Probability (RSI = {rsi_fixed})')
plt.grid(True, alpha=0.3)
plt.xlim(0.2, 1.0)
plt.ylim(0, 1)
plt.show()

In [ ]:
# Generate synthetic validation data
np.random.seed(42)
n_samples = 1000

# Simulate RSI values
rsi_sim = np.random.beta(2, 2, n_samples)

# Calculate true probabilities
calibrator = ProbabilityCalibrator('pinus_halepensis')
p_true = [calibrator.calibrate_probability(r, 0.9) for r in rsi_sim]

# Generate binary outcomes
outcomes = np.random.binomial(1, p_true)

# Create dataframe
df = pd.DataFrame({
    'RSI': rsi_sim,
    'Probability': p_true,
    'Rapid_Spread': outcomes
})

print(df.head())

In [ ]:
# Reliability diagram
bins = np.linspace(0, 1, 11)
bin_centers = (bins[:-1] + bins[1:]) / 2
observed_freq = []

for i in range(len(bins)-1):
    mask = (df['Probability'] >= bins[i]) & (df['Probability'] < bins[i+1])
    if mask.sum() > 0:
        observed_freq.append(df[mask]['Rapid_Spread'].mean())
    else:
        observed_freq.append(np.nan)

plt.figure(figsize=(10, 6))
plt.plot(bin_centers, observed_freq, 'bo-', linewidth=2, label='Observed')
plt.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect reliability')
plt.xlabel('Forecast Probability')
plt.ylabel('Observed Frequency')
plt.title('Reliability Diagram')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

In [ ]:
# ROC Curve
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(df['Rapid_Spread'], df['Probability'])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Random classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

print(f"AUC: {roc_auc:.3f}")

In [ ]:
# Complete example with forecaster
forecaster = RapidSpreadForecaster('pinus_halepensis')

result = forecaster.predict(
    lfm=65,
    dfm=6,
    cbd=0.18,
    sfl=45,
    fbd=0.8,
    wind_speed=10.4,
    vpd=38.1,
    aspect=225,
    drought_code=487
)

print("\n=== Forecast Results ===")
print(f"Rapid Spread Index (RSI): {result['rsi']:.3f}")
print(f"Probability: {result['probability']:.1%}")
print(f"Confidence: {result['confidence']:.1%}")
print(f"Hazard Level: {result['hazard_level'].upper()}")
print(f"Lead Time: {result['lead_time']} minutes")

In [ ]:
print("\n✅ Probability calibration completed successfully")